In [1]:
from pvrecorder import PvRecorder
from scipy.fft import fft, fftfreq
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import time 
for index, device in enumerate(PvRecorder.get_available_devices()):
    print(f"[{index}] {device}")


[0] Discard all samples (playback) or generate zero samples (capture)
[1] Rate Converter Plugin Using Libav/FFmpeg Library
[2] Rate Converter Plugin Using Samplerate Library
[3] Rate Converter Plugin Using Speex Resampler
[4] JACK Audio Connection Kit
[5] Open Sound System
[6] PulseAudio Sound Server
[7] Plugin for channel upmix (4,6,8)
[8] Plugin for channel downmix (stereo) with a simple spacialization
[9] USB Device 0x46d:0x825, USB Audio


In [2]:
class MicrophoneReaderBuffer:
    def __init__(self, device_index, frame_length, signal_nframe, sxx_nframe):

        # recorder 
        recorder = PvRecorder(frame_length=frame_length, device_index=device_index)
        recorder.start()
        self.recorder  = recorder 

        # values 
        freqs = fftfreq(recorder.frame_length*signal_nframe, 1/recorder.sample_rate)
        n_freqs = int(len(freqs) //2)
        freqs = freqs[:n_freqs]
        frame_duration = recorder.frame_length / recorder.sample_rate

        # buffer 
        self.signal_buffer = [np.zeros(frame_length) for _ in range(signal_nframe)]
        self.sxx_buffer = [np.zeros(n_freqs) for _ in range(sxx_nframe)]


        # values 
        self.freqs = freqs
        self.n_freqs = n_freqs
        self.frame_length = frame_length
        self.frame_duration = frame_duration


    def sample(self):

        x = self.recorder.read()
        
        self.signal_buffer = self.signal_buffer[1:] + [x]

        y = fft(np.concatenate(self.signal_buffer))[:self.n_freqs]
        sxx = np.log(y.real**2 + y.imag**2)
        
        self.sxx_buffer =  self.sxx_buffer[1:] + [sxx] 



In [3]:
def running_spectrogram(freqs, total_frames=200):


    data = [np.zeros(len(freqs)) for _ in range(total_frames)]
    heatmap_fig = go.FigureWidget([go.Heatmap( y = freqs, zmin=0, zmax=6)])


    def update_figure(sxx_frames):
        data.pop(0)
        data.append(sxx_frames)
    
        heatmap_fig.data[0].z = np.stack(data, axis=-1)

    return heatmap_fig, update_figure

def running_spectro_slice(freqs):

    line_fig = go.FigureWidget([go.Scatter( x = freqs)])

    def update_figure(value):
        line_fig.data[0].y = value

    return line_fig, update_figure

def get_updating_table(columns):
    fig = go.FigureWidget(go.Table(
        header=dict(values=columns),
        cells=dict(values=[[], []])
    ))

    def update_table(data_dict):
        """
        take 


        fig, update_table  = get_updating_table(['a', 'b'])
        update_table({'a':[1,2,3], 'b':[2,4,5]})
        """
        values = []
        for c in columns:
            values.append(data_dict[c])
        
        fig.data[0].cells.values = values
    return fig, update_table

def get_freq_bands(values, freqs, threshold, peak_threshold):

    values = np.array(values)
    freqs = np.array(freqs)

    bands = [[]]
    for idx, v in enumerate(values):
        if v > threshold:
            bands[-1].append(idx)
        elif len(bands[-1]):
            bands.append([])

    bands = [i for i in bands if len(i)>0 and len(i)<80]

    freq_centres = []
    freq_peak_powers = []
    for each_band in bands:
        freq_peak_power = values[each_band].max()
        if freq_peak_power < peak_threshold:
            continue
        

        centre_idx_idx = np.argmax(values[each_band])
        centre_idx = each_band[centre_idx_idx]
        freq_centre = freqs[centre_idx]
        

        freq_centres.append(freq_centre)
        freq_peak_powers.append(freq_peak_power)

    return freq_centres, freq_peak_powers



def normalise(values):
    median = np.median(values)
    mad = np.median(abs(values-median))

    return (values - median)/mad



def find_peaks(normalised, freqs):


    freq_centres, freq_peak_powers = get_freq_bands(normalised, freqs, 2, 4)

    #pitches = [freq_to_pitch(f) for f in freq_centres]

    
    freq_centres = np.array([0]+[f for f in freq_centres if f > 40])

    basefreq = np.nan
    if len(freq_centres)>3:
        gap1 = freq_centres[1:] - freq_centres[:-1]
        #gap2 = freq_centres[2:] - freq_centres[:-2]

        basefreq = np.median(np.concatenate([gap1]))
        

    return basefreq, freq_centres, freq_peak_powers


def freq_to_pitch(freq):
    return (np.log2(freq) % 1) * 360


def find_peak_indices(values, threshold, eps=0.5):
    """
    both rising and falling edge need to be above a threshold
    """
    prev = values[0]
    tough = values[0]
    peak = values[0]
    rising_edge_length = 0

    rising = True
    bands = []
    for idx, curr in enumerate(values[1:]):

        # end of rising edge
        if rising and (prev-curr)>0:
            rising = False 
            peak = prev + eps
            peak_idx = idx - 1 + 1 # started from index 1
            rising_edge_length = peak - tough

        # end of falling edge
        if (not rising) and (curr-prev)>0:
            rising = True
            tough = prev - eps
        
            falling_edge_length = peak - tough 
            if (rising_edge_length > threshold) & (falling_edge_length>threshold):
                bands.append(peak_idx)
        
        prev = curr - eps if rising else curr + eps

    return bands

def find_basefreq(freqs):

    freqs = [0] + [f for f in freqs if f > 40]
    freqs = np.array(freqs)
    basefreq = np.nan

    if len(freqs)>3:
        gap1 = freqs[1:] - freqs[:-1]
        #gap2 = freqs[2:] - freqs[:-2]

        gaps = np.sort(np.concatenate([gap1]))


        basefreq = gaps[(len(gaps)+1)//2]
        
    return basefreq

In [11]:
0 and 12%0

0

In [4]:

readerBuffer = MicrophoneReaderBuffer(9, 800, 5, 100)


table, table_update = get_updating_table(['frequency', 'strength'])
spectro_fig, spectro_update = running_spectrogram(readerBuffer.freqs)
slice_fig, slice_update = running_spectro_slice(readerBuffer.freqs)

In [9]:
i = 0
while 1:
    readerBuffer.sample()

    if  (not i % 10):
        
        slice = np.stack(readerBuffer.sxx_buffer[-3:], axis=-1).mean(-1)
        slice = normalise(slice[np.where(readerBuffer.freqs < 2000) ])
        
        #spectro_update(slice)        

        if not (i%25):
            slice_update(slice)

        #indices = find_peak_indices(slice, 5)

        #freqs = readerBuffer.freqs[indices]
        #strengths = slice[indices]
        #basefreq = find_basefreq(freqs)
        #pitch, freq, strength = find_peaks(slice, readerBuffer.freqs)

        freqs, strengths = get_freq_bands(slice, readerBuffer.freqs, 2, 4)

        basefreq = find_basefreq(np.array(freqs))

        print(basefreq, end='          \r')
        table_update(dict(
            frequency = freqs, 
            strength = strengths, 
        ))
    
    
    i+=1


KeyboardInterrupt: 

In [8]:
table

FigureWidget({
    'data': [{'cells': {'values': [[0.0], [5.289845079932258]]},
              'header': {'values': ['frequency', 'strength']},
              'type': 'table',
              'uid': '00311243-62ee-4f4a-a277-76c02bf75e1d'}],
    'layout': {'template': '...'}
})

In [7]:
slice_fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'e099f0c9-e403-4737-9427-daec5bb240c9',
              'x': array([0.000e+00, 4.000e+00, 8.000e+00, ..., 7.988e+03, 7.992e+03, 7.996e+03]),
              'y': array([ 5.20076176,  0.01403737,  0.30337646, ..., -1.75371409, -1.38380922,
                          -1.7005456 ])}],
    'layout': {'autosize': True, 'template': '...'}
})

In [ ]:
spectro_fig